In [1]:
import pandas as pd
from sklearn.utils import shuffle
import os
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import os
from skimage import io
import torch
from torchvision import transforms
import torchvision
from skimage import color
import copy

import time
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models
from torchvision import transforms as T
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import pandas as pd
from skimage import io, transform
import matplotlib.image as mpimg
from PIL import Image
from sklearn.metrics import roc_auc_score
import torch.nn.functional as F
import scipy
import random
import pickle
import scipy.io as sio
import itertools
from scipy.ndimage.interpolation import shift
import copy
import warnings
#warnings.filterwarnings("ignore")
plt.ion()
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

## 1. Prepare Train/Valid/Test Set

In [4]:
os.getcwd()

'/Users/nhungle/Box/Free/Data-Science-Projects/Breast_Cancer_Diagnosis'

In [127]:
excel_path = '/Users/nhungle/Box/Free/Data-Science-Projects/Breast_Cancer_Diagnosis/excel_files'

In [128]:
cal_test = pd.read_csv(os.path.join(excel_path, 'calc_case_description_test_set.csv'))
cal_train = pd.read_csv(os.path.join(excel_path, 'calc_case_description_train_set.csv'))
mass_test = pd.read_csv(os.path.join(excel_path, 'mass_case_description_test_set.csv'))
mass_train = pd.read_csv(os.path.join(excel_path, 'mass_case_description_train_set.csv'))

In [134]:
cal_test = cal_test[['patient_id','left or right breast', 'image view',
            'pathology', 'image file path',
        'cropped image file path', 'ROI mask file path']]
mass_test = mass_test[['patient_id','left or right breast', 'image view',
            'pathology', 'image file path',
        'cropped image file path', 'ROI mask file path']]
cal_train = cal_train[['patient_id','left or right breast', 'image view',
        'pathology', 'image file path',
    'cropped image file path', 'ROI mask file path']]
mass_train = mass_train[['patient_id','left or right breast', 'image view',
        'pathology', 'image file path',
    'cropped image file path', 'ROI mask file path']]

In [10]:
cal_test.dropna(axis = 0, how = 'all', inplace = True)
mass_test.dropna(axis = 0, how = 'all', inplace = True)
testSet = pd.concat([cal_test, mass_test], axis=0)
train_valid = pd.concat([cal_train, mass_train], axis=0)

In [11]:
train_valid['class'] = 0
train_valid.loc[train_valid['pathology'] == 'MALIGNANT', 'class'] = 1
testSet['class'] = 0
testSet.loc[testSet['pathology'] == 'MALIGNANT', 'class'] = 

In [13]:
patient_list = list(train_valid['patient_id'].unique())
print('Number of patients: {}'.format(len(patient_list)))

Number of patients: 1248


In [14]:
unique_id_leftright = train_valid.groupby(['patient_id', 'left or right breast']).size().reset_index().rename(columns={0:'count'})
unique_id_leftright = shuffle(unique_id_leftright)
train_size = int(len(unique_id_leftright) * 0.8)
#print(train_size)
train_id_leftright = unique_id_leftright[:train_size][['patient_id', 'left or right breast']]
valid_id_leftright = unique_id_leftright[train_size:][['patient_id', 'left or right breast']]

In [15]:
trainSet = train_valid[(train_valid['patient_id'].isin(train_id_leftright['patient_id'])) &
            (train_valid['left or right breast'].isin(train_id_leftright['left or right breast'])) ]

In [16]:
validSet = train_valid[(train_valid['patient_id'].isin(valid_id_leftright['patient_id'])) &
            (train_valid['left or right breast'].isin(valid_id_leftright['left or right breast'])) ]

In [58]:
# Write to CSV
trainSet.to_csv(os.path.join(excel_path, 'twoClass_trainSet.csv'))
validSet.to_csv(os.path.join(excel_path, 'twoClass_validSet.csv'))
testSet.to_csv(os.path.join(excel_path, 'twoClass_testSet.csv'))

In [152]:
# Get a subset of data 
trainSet_sample = trainSet[:500]
valSet_sample = validSet[:100]
testSet_sample = testSet[:100]

In [154]:
trainSet_sample.to_csv(os.path.join(excel_path, 'twoClass_trainSet_sample.csv'))
valSet_sample.to_csv(os.path.join(excel_path, 'twoClass_validSet_sample.csv'))
testSet_sample.to_csv(os.path.join(excel_path, 'twoClass_testSet_sample.csv'))

In [18]:
class MamogramDataset(Dataset):

    def __init__(self, csv_file, root_dir, image_column, transform=None):
        """
        Args:
            csv_file (string): Csv file filename information.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
            image_column (string): name of the column image used
        """
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.image_column = image_column
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.data_frame.loc[idx, 'image_column'])

        image = io.imread(img_name,as_gray=True)
        
        image = (image - image.mean()) / image.std()
            
        image_class = self.data_frame.loc[idx, 'Class']

        sample = {'x': image[None,:], 'y': image_class}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [122]:
def GetDataLoader(train_csv, validation_csv, test_csv, root_dir, image_column,
               train_transform, validation_transform, 
               batch_size, shuffle, num_workers): 

    train_data = MamogramDataset(csv_file = train_csv, 
                              root_dir = root_image,
                              image_column = image_column,
                               transform=train_transform)
    val_data = MamogramDataset(csv_file = validation_csv, 
                            root_dir = root_image,
                            image_column = image_column,
                            transform = validation_transform)
    test_data = MamogramDataset(csv_file = test_csv, 
                            root_dir = root_image,
                            image_column = image_column,
                            transform = validation_transform)
    
    image_datasets = {'train': train_data, 'val': val_data, 'test': test_data}
#     train_loader = DataLoader(train_data, batch_size=BATCH_SIZE,
#                             shuffle = shuffle, num_workers = NUM_WORKERS)
#     val_loader = DataLoader(val_data, batch_size=BATCH_SIZE,
#                             shuffle = shuffle, num_workers = NUM_WORKERS)
#     test_loader = DataLoader(test_data, batch_size=BATCH_SIZE,
#                             shuffle = shuffle, num_workers = NUM_WORKERS)
    

    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                              batch_size=BATCH_SIZE, 
                                              shuffle=True, 
                                              num_workers=NUM_WORKERS) 
                    for x in ['train', 'val', 'test']}
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
#     print(len(image_datasets['train']), 
#           len(image_datasets['val']),
#          len(image_datasets['test']))
    return dataloaders, dataset_sizes

In [126]:
def train_model(model, criterion, optimizer, scheduler, num_epochs = 10,verbose = True):

    start_time = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    acc_dict = {'train':[],'validation':[]}
    loss_dict = {'train':[],'validation':[]}

    for epoch in range(num_epochs):
        if verbose:
            if epoch % 5 == 4:
                print('Epoch {}/{}'.format(epoch+1, num_epochs))
                print('-' * 10)
            
        for phase in ['train','val']:
            if phase == 'train':
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0
            running_corrects = 0

            for data in dataloaders[phase]:
                
                inputs = data['x']
                labels = data['y']
                
                # wrap them in Variable
                if use_gpu:
                    inputs = inputs.type(torch.FloatTensor).to(device)
                    labels = labels.to(device)
                else:
                    inputs = Variable(inputs).type(torch.FloatTensor)
                    labels = Variable(labels).type(torch.LongTensor)

                    
                optimizer.zero_grad()

                out = model(inputs)
                _, preds = torch.max(out, dim = 1)
                loss = criterion(out, labels)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                    
                running_loss += loss.item() * inputs.size()[0]
                running_corrects += torch.sum(preds == labels).item()
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            if verbose:
                if epoch % 5 == 4:
                    print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'train':
                loss_dict['train'].append(epoch_loss)
                acc_dict['train'].append(epoch_acc)
            else:
                loss_dict['validation'].append(epoch_loss)
                acc_dict['validation'].append(epoch_acc)
                    
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                scheduler.step(epoch_loss)

    time_elapsed = time.time() - start_time
    print('Training time: {}minutes {}s'.format(int(time_elapsed / 60), time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    # for i, phase in enumerate(['train','validation']):

    #     fig = plt.figure()
        
    #     a = fig.add_subplot(1,2,1*i+1)
    #     plt.plot(loss_dict[phase])
    #     plt.title('Loss per epoch for ' + phase)

    #     a = fig.add_subplot(1,2,1*i+2)
    #     plt.plot(acc_dict[phase])
    #     plt.title('Accuracy per epoch for ' + phase)
    #     plt.show()
    #     plt.savefig(os.path.join(graph_path ,'Curve {}.png'.format(phase)))

    model.load_state_dict(best_model_wts)
    torch.save(model, os.path.join(graph_path, 'BestResNet34_tl.pt'))
    
    return {'Model': model, 'LossDict': loss_dict, 'AccDict': acc_dict}

In [123]:
# ######### Local Machine Paths ######## 
excel_path = '/Users/nhungle/Box/Free/Data-Science-Projects/Breast_Cancer_Diagnosis/excel_files'
train_local_csv = os.path.join(excel_path, 
                              'train_local.csv')
validation_local_csv = os.path.join(excel_path, 
                              'validation_local.csv')
test_local_csv = os.path.join(excel_path, 
                              'test_local.csv')

image_path = '/Users/nhungle/Box/Free/Data-Science-Projects/Breast_Cancer_Diagnosis'
root_image = os.path.join(image_path ,'images')

NUM_WORKERS = 1
BATCH_SIZE = 1
graph_path = '/Users/nhungle/Box/Free/Data-Science-Projects/Breast_Cancer_Diagnosis/graphs'


In [ ]:
# ######### HPC Paths ######## 
# excel_path = '/home/nhl256/BreastCancer/excel_files'
# train_local_csv = os.path.join(excel_path, 
#                              'trainSet.csv')
# validation_local_csv = os.path.join(excel_path, 
#                               "validationSet.csv")
# test_local_csv = os.path.join(excel_path, 
#                               "testSet.csv")

# image_path = '/scratch/bva212/breastCancerData'
# root_image = os.path.join(image_path ,'CBIS-DDSM')

# NUM_WORKERS = 4
# BATCH_SIZE = 16
# graph_path = '/home/nhl256/HW2'


In [124]:
use_gpu = torch.cuda.is_available()
if torch.cuda.is_available:
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

#### Get Dataloaders and Datasets_sizes
dataloaders, dataset_sizes = GetDataLoader(train_csv = train_local_csv, 
                                            validation_csv = validation_local_csv, 
                                            test_csv = test_local_csv, 
                                            root_dir = root_image, 
                                           image_column = 'local_image',
               train_transform = None, validation_transform = None, 
               batch_size = BATCH_SIZE, shuffle = True, num_workers = NUM_WORKERS) 

In [147]:
#pd.read_csv(os.path.join(excel_path, 'train_local.csv'))

### Prepare train/validatin/test local set to run code locally

In [53]:
def getImageName(image_string): 
    return image_string.split('/')[3].split('-')[-1] + '.dcm'

In [70]:
trainSet['local_image'] = ''
trainSet['local_image'] = trainSet['image file path'].apply(lambda x: getImageName(x))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [115]:
images_files = ['00446.dcm', '32980.dcm', '38548.dcm', '44292.dcm', '64189.dcm', '64555.dcm', 
         '68297.dcm', '83789.dcm', '96727.dcm', '97907.dcm']
local_frame = trainSet[trainSet['local_image'].isin(images_files)]

local_frame

,patient_id,left or right breast,image view,pathology,image file path,cropped image file path,ROI mask file path,class,local_image
0,P_00005,RIGHT,CC,MALIGNANT,CBIS-DDSM/Calc-Training_P_00005_RIGHT_CC/08-07...,CBIS-DDSM/Calc-Training_P_00005_RIGHT_CC_1/08-...,CBIS-DDSM/Calc-Training_P_00005_RIGHT_CC_1/08-...,1,38548.dcm
1,P_00005,RIGHT,MLO,MALIGNANT,CBIS-DDSM/Calc-Training_P_00005_RIGHT_MLO/08-0...,CBIS-DDSM/Calc-Training_P_00005_RIGHT_MLO_1/09...,CBIS-DDSM/Calc-Training_P_00005_RIGHT_MLO_1/09...,1,96727.dcm
2,P_00007,LEFT,CC,BENIGN,CBIS-DDSM/Calc-Training_P_00007_LEFT_CC/08-07-...,CBIS-DDSM/Calc-Training_P_00007_LEFT_CC_1/09-0...,CBIS-DDSM/Calc-Training_P_00007_LEFT_CC_1/09-0...,0,68297.dcm
3,P_00007,LEFT,MLO,BENIGN,CBIS-DDSM/Calc-Training_P_00007_LEFT_MLO/08-07...,CBIS-DDSM/Calc-Training_P_00007_LEFT_MLO_1/09-...,CBIS-DDSM/Calc-Training_P_00007_LEFT_MLO_1/09-...,0,00446.dcm
5,P_00008,LEFT,MLO,BENIGN_WITHOUT_CALLBACK,CBIS-DDSM/Calc-Training_P_00008_LEFT_MLO/08-07...,CBIS-DDSM/Calc-Training_P_00008_LEFT_MLO_1/09-...,CBIS-DDSM/Calc-Training_P_00008_LEFT_MLO_1/09-...,0,97907.dcm
7,P_00008,RIGHT,MLO,BENIGN_WITHOUT_CALLBACK,CBIS-DDSM/Calc-Training_P_00008_RIGHT_MLO/08-0...,CBIS-DDSM/Calc-Training_P_00008_RIGHT_MLO_1/09...,CBIS-DDSM/Calc-Training_P_00008_RIGHT_MLO_1/09...,0,32980.dcm


In [116]:
train_local = local_frame[:4]
validation_local = local_frame[4:5]
test_local = local_frame[5:6]

In [117]:
train_local.to_csv(os.path.join(excel_path, 'train_local.csv'))
validation_local.to_csv(os.path.join(excel_path, 'validation_local.csv'))
test_local.to_csv(os.path.join(excel_path, 'test_local.csv'))

In [118]:
train_temp = pd.read_csv(os.path.join(excel_path, 'train_local.csv'))
train_temp.shape

(4, 10)